In [63]:
import requests
from datetime import datetime, UTC
from langchain.agents import tool
from pydantic import BaseModel, Field

latitude = -23.6865
longitude = -46.6234


class RetornaTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')

@tool(args_schema=RetornaTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    '''Retorna a temperatura atual para cada coordenada'''
    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly" : "temperature_2m",
        'forecast_days': 1
    }
    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()

        hora_agora = datetime.now(UTC).replace(tzinfo=None)
        lista_horas = [datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x]-hora_agora))
        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]

        return temp_atual
    else:
        raise Exception(f'Request para API {URL} falhou: {resposta.status_code}')
    

In [65]:
retorna_temperatura_atual.args

{'latitude': {'description': 'Latitude da localidade que buscamos a temperatura',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude da localidade que buscamos a temperatura',
  'title': 'Longitude',
  'type': 'number'}}

In [66]:
retorna_temperatura_atual.invoke({'latitude' : latitude, 'longitude': longitude})

20.5

In [84]:
import wikipedia
wikipedia.set_lang('pt')

query = 'Isaac Asimov'

class search_wiki(BaseModel):
    query : str = Field(description='Pergunta ou assunto que deve ser localizado no wikipedia')

@tool(args_schema=search_wiki)
def busca_wikipedia(query:str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query feita"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo}\nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return 'Busca não teve retorno'
    else:
        return '\n\n'.join(resumos)

busca_wikipedia.invoke({'query':'Isaac ASIMOV'})

'Título da página: Isaac Asimov\nResumo: Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.\nAsimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.\nA maioria de seus livro

## Testando a Tool

In [88]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'Você é um assistente amigável chamado Joselito, expert em conhecimentos gerais.'),
        ('user','{input}')
    ]
)

chat = ChatOpenAI()

tools = [convert_to_openai_function(busca_wikipedia), convert_to_openai_function(retorna_temperatura_atual)]
chain = prompt | chat.bind(functions=tools)

In [89]:
chain.invoke({'input':'Olá'})

AIMessage(content='Olá! Como posso te ajudar hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 151, 'total_tokens': 164, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-702f2540-7226-4d20-9aba-4f6e8beb4ca9-0', usage_metadata={'input_tokens': 151, 'output_tokens': 13, 'total_tokens': 164, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [90]:
chain.invoke({'input':'Qual é a temperatura atual de diadema?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-23.6819,"longitude":-46.6228}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 159, 'total_tokens': 188, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-8cbe5334-f71a-473c-8012-4ca1933a0938-0', usage_metadata={'input_tokens': 159, 'output_tokens': 29, 'total_tokens': 188, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [91]:
chain.invoke({'input':'Quem foi Isaac Newton?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Isaac Newton"}', 'name': 'busca_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 155, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-8e442e83-521a-446e-b94c-26727710a00c-0', usage_metadata={'input_tokens': 155, 'output_tokens': 19, 'total_tokens': 174, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})